In [1]:
import pandas as pd
import sqlite3
import logging
from ingestion_db import ingest_db
import time

logging.basicConfig(
    filename="logs/vendor_sales_summary.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(filename)s - %(message)s",
    filemode='a'
)


def create_vendor_summary(conn):
    '''This function will merge different tables to get overall summary and adding multiple columns in the resulting data'''
    vendor_sales_summary = pd.read_sql_query("""WITH FreightSummary AS(
        SELECT
            VendorNumber,
            SUM(freight) as TotalFreightCost
        FROM vendor_invoice
        GROUP BY VendorNumber
    ),
    
    PurchaseSummary AS(
        SELECT
            p.VendorNumber,
            p.VendorName,
            p.Brand,
            p.Description,
            p.PurchasePrice,
            pp.Price as ActualPrice,
            pp.Volume,
            SUM(p.Quantity) AS TotalPurchaseQuantity,
            SUM(p.Dollars) AS TotalPurchaseDollars
        FROM purchases p
        JOIN purchase_prices pp
            ON p.Brand = PP.Brand
        WHERE p.PurchasePrice>0
        GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
    ),
    
    SalesSummary AS(
        SELECT
            VendorNo,
            Brand,
            SUM(SalesQuantity) AS TotalSalesQuantity,
            SUM(SalesDollars) AS TotalSalesDollars,
            SUM(SalesPrice) AS TotalSalesPrice,
            SUM(ExciseTax) AS TotalExciseTax
        FROM sales
        GROUP BY VendorNo, Brand
    )
    
    SELECT
        ps.VendorNumber,
        ps.VendorName,
        ps.Brand,
        ps.Description,
        ps.PurchasePrice,
        ps.ActualPrice,
        ps.Volume,
        ps.TotalPurchaseQuantity,
        ps.TotalPurchaseDollars,
        ss.TotalSalesQuantity,
        ss.TotalSalesDollars,
        ss.TotalSalesPrice,
        ss.TotalExciseTax,
        fs.TotalFreightCost
    FROM PurchaseSummary ps
    LEFT JOIN SalesSummary ss
        ON ps.VendorNumber=ss.VendorNo
        AND ps.Brand=ss.Brand
    LEFT JOIN FreightSummary fs
        ON ps.VendorNumber=fs.VendorNumber
    ORDER BY ps.TotalPurchaseDollars DESC""", conn)

    return vendor_sales_summary

def clean_data(df):
    '''this function will clean data'''
    #changing datatype to float
    df['Volume'] = df['Volume'].astype('float64')

    #filling missing value to 0
    df.fillna(0, inplace=True)

    #removing spaces from categorical columns
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    #creating new columns for better analysis
    df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']
    df['GrossMargin'] = (df['GrossProfit']/df['TotalSalesDollars']) *100
    df['StockTurnOver'] = df['TotalSalesQuantity']/df['TotalPurchaseQuantity']
    df['SalesToPurchaseRatio'] = df['TotalSalesDollars']/df['TotalSalesDollars']

    return df
    
if __name__=='__main__':
    #creating database connection
    conn = sqlite3.connect('inventory.db')

    logging.info('Creating vendor summary table......')
    summary_df=create_vendor_summary(conn)
    logging.info(summary_df.head())

    logging.info('Cleaning Data.....')
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())
    
    logging.info('ingesting Data......')
    ingest_db(clean_df, 'vendor_sales_summary', conn)
    logging.info('completed')

    
